# Estimating an economical heating switchover point for dual fuel heat pump/furnace HVAC systems

I recently had an air source heat pump[^heatpump-bg] installed at our house, but I also left our existing high-efficiency natural gas furnace installed. This required me to decide when to switch between those two heating fuels (electricity vs. natural gas). There are multiple possible factors to consider when deciding on fuels, one of which is the economy of each fuel. I had to piece together information and guidance from multiple sources to arrive at an answer, so I collected links to those resources and the computations that I ultimately performed in case they may be informative to others. *However, I am not an HVAC professional, and anything in this notebook may be incorrect. Use at your own risk/expense!*

The crucial difference between an air conditioner and an air source heat pump is that a heat pump is able to reverse its operation and pump heat from outdoor air to indoor air. Moving heat in this way is fundamentally different from converting chemical energy into heat, e.g., by burning natural gas, or converting electricity into heat via resistive heating. A heat pump is more efficient than resistive heating throughout its operational range[^csu-dual-fuel-wp], and both have the same per-unit fuel cost. On the other hand, gas may have a different cost than electricity for an equivalent amount of raw energy. A heat pump becomes less efficient as the temperature difference between the indoor and outdoor air increases, since it needs to work harder to transfer a given amount of heat into indoor air. At a certain point as that difference increases due to outdoor temperatures dropping while the indoor heating setpoint remains roughly constant, it may become more economical to burn natural gas than it is to run a heat pump.

The CSU whitepaper (footnote 4) explains how to compute that switchover point for minimizing combined fuel costs, but it depends on the curve for the Coefficient of Performance (COP) of the heat pump being known. The COP is a rating of how efficiently the heat pump can transfer heat, and it varies with temperature for the reasons noted above. An HVAC contractor provided advice online for obtaining or estimating COP ratings[^obtaining-cop].

For simplicity, and due to my lack of a detailed COP curve for my particular heat pump, I assumed a linear COP curve. This is certainly inaccurate, but at least it's better than picking an arbitrary switchover point. Perhaps this could be made more accurate even in cases where only a couple of COP data points are available by assuming that the same quadratic curve applies to a particular heat pump as was fitted to a large number of data points in a study[^cop-curve]. However, the independent variable in that study is the temperature difference between the indoor and outdoor air, whereas the commonly reported COP samples are stated for a couple of absolute outdoor temperatures (specifically, for 17 and 47 degrees Fahrenheit). I have not seen what indoor temperature setpoint is assumed for those data points, but that information is probably available somewhere.

The calculations below are what I used to compute the approximate most economical fuel switchover point for my particular HVAC system. The constants will vary across systems.

Some electric rate plans offer varying electricity prices throughout the day, which would lead to a corresponding variation in the relative economy of different fuels. I am unsure whether any thermostats offer the ability to automatically change the switchover point on a schedule, either directly in the thermostat interface or via an API. That's something I may investigate more in the future.

Of course, fuel prices also change over longer periods of time, even for utility rate plans that are constant throughout a particular day. Thus, I'll need to update the constants and recompute this periodically.

[^heatpump-bg]: For basic background on heat pumps, see https://www.cnet.com/home/energy-and-utilities/heat-pump-vs-furnace-learn-the-difference-between-these-two-heat-sources/
[^csu-dual-fuel-wp]: Colorado Springs Utilities, "DUAL FUEL GAS HEAT & AIR SOURCE HEAT PUMP WHITE PAPER #30", https://www.csu.org/Documents/DualFuelGasHeat.pdf
[^obtaining-cop]: Adams Air, "UNDERSTANDING COP (COEFFICIENT OF PERFORMANCE)", https://www.adams-air.com/houston/what-is-COP.php
[^cop-curve]: Oliver Ruhnau, Lion Hirth, and Aaron Praktiknjo, Figure 6 from "Time series of heat demand and heat pump efficiency for energy system modeling", https://www.nature.com/articles/s41597-019-0199-y/figures/6

In [2]:
# Coefficient Of Performance (COP) for heat pump at 17 degrees Fahrenheit:
cop_17 <- 2.56
# Coefficient Of Performance (COP) for heat pump at 47 degrees Fahrenheit:
cop_47 <- 3.72
# Cost of gas in terms of conventional billing units (e.g., can be computed from natural gas bill):
cost_gas_per_therm <- 1.06 # money per therm
# Cost of electricity in terms of conventional billing units (e.g., can be computed from electric bill):
cost_elec_per_kwh <- 0.125 # money per kWh
# Unit costs converted to BTUs:
cost_gas_per_btu <- cost_gas_per_therm / 100000 # Conversion factor from https://www.eia.gov/tools/faqs/faq.php?id=45&t=8
cost_elec_per_btu <- cost_elec_per_kwh / 3412 # Conversion factor from https://www.eia.gov/energyexplained/units-and-calculators/energy-conversion-calculators.php
# Efficiency of gas furnace (e.g., 0.80 for 80% efficiency or 0.90 for 90% efficiency):
gas_furnace_efficiency <- 0.90
# Threshold for the COP of the heat pump at which it becomes more economical than the gas furnace,
# from rearranging the equation on page 6 of https://www.csu.org/Documents/DualFuelGasHeat.pdf:
cop_threshold <- cost_elec_per_btu / (cost_gas_per_btu * (1/gas_furnace_efficiency))
# Compute temperature at which the COP threshold is reached assuming (inaccurately) a linear COP curve:
cop_slope <- (cop_47 - cop_17) / (47 - 17)
cop_y_intercept <- cop_17 - cop_slope * 17
temp_for_cop_threshold <- (cop_threshold - cop_y_intercept) / cop_slope
print(temp_for_cop_threshold)

[1] 31.23845
